In [1]:
import torch
from geom2vec.data import create_model, infer_traj
device = "cuda" if torch.cuda.is_available() else "cpu"
rep_model = create_model(
    model_type = 'tn',
    checkpoint_path = '../../checkpoints/tensornet_l3_h128_rbf32_r5.pth',
    cutoff = 5,
    hidden_channels = 128,
    num_layers = 3,
    num_rbf = 32,
    device = device
)

/beagle3/dinner/scguo/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/beagle3/dinner/scguo/envs/torch/lib/python3.11/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /beagle3/dinner/scguo/envs/torch/lib/python3.11/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/beagle3/dinner/scguo/envs/torch/lib/python3.11/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /beagle3/dinner/scguo/envs/torch/lib/python3.11/site-packages/libpyg.so)
  warnings.warn(f"An issue

In [2]:
from geom2vec.data import extract_mda_info, extract_mda_info_folder

topology_file = "/project/dinner/anton_trajs/Chignolin/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/chignolin.pdb"
trajectory_folder = "/project/dinner/anton_trajs/Chignolin/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein" 

# you should carefully check the output to make sure the 
position_list, atomic_numbers, segment_counts, dcd_files = extract_mda_info_folder(
    folder = trajectory_folder,
    top_file = topology_file,
    stride = 10,
    selection = 'prop mass > 1.1', # by default, we only consider heavy atoms, this is the default selection
)

Processing CLN025-0-protein-000.dcd


/beagle3/dinner/scguo/envs/torch/lib/python3.11/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


Processing CLN025-0-protein-001.dcd
Processing CLN025-0-protein-002.dcd
Processing CLN025-0-protein-003.dcd
Processing CLN025-0-protein-004.dcd
Processing CLN025-0-protein-005.dcd
Processing CLN025-0-protein-006.dcd
Processing CLN025-0-protein-007.dcd
Processing CLN025-0-protein-008.dcd
Processing CLN025-0-protein-009.dcd
Processing CLN025-0-protein-010.dcd
Processing CLN025-0-protein-011.dcd
Processing CLN025-0-protein-012.dcd
Processing CLN025-0-protein-013.dcd
Processing CLN025-0-protein-014.dcd
Processing CLN025-0-protein-015.dcd
Processing CLN025-0-protein-016.dcd
Processing CLN025-0-protein-017.dcd
Processing CLN025-0-protein-018.dcd
Processing CLN025-0-protein-019.dcd
Processing CLN025-0-protein-020.dcd
Processing CLN025-0-protein-021.dcd
Processing CLN025-0-protein-022.dcd
Processing CLN025-0-protein-023.dcd
Processing CLN025-0-protein-024.dcd
Processing CLN025-0-protein-025.dcd
Processing CLN025-0-protein-026.dcd
Processing CLN025-0-protein-027.dcd
Processing CLN025-0-protein-

In [3]:
import MDAnalysis as mda

topology_file = "/project/dinner/anton_trajs/Chignolin/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/chignolin.pdb"
trajectory_file = "/project/dinner/anton_trajs/Chignolin/DESRES-Trajectory_CLN025-0-protein/CLN025-0-protein/CLN025-0-protein-000.dcd" 

u = mda.Universe(topology_file, trajectory_file)
# number of atoms
print(len(u.atoms))
# exclude hydrogens
print(len(u.select_atoms('prop mass > 1.1')))


166
93


In [4]:
print(f'length of position_list: {len(position_list)}')
print('position_list[0].shape:', position_list[0].shape)
print('atomic_numbers:', atomic_numbers.shape)

length of position_list: 54
position_list[0].shape: (1000, 93, 3)
atomic_numbers: (93,)


In [5]:
# create a folder to store the results
import os

folder_path = '/project/dinner/zpengmei/geom2vec/results/chignolin_stride10'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# infer the trajectory
infer_traj(
    model = rep_model,
    hidden_channels = 128,
    batch_size = 100,
    data = position_list,
    atomic_numbers = atomic_numbers,
    cg_mapping = segment_counts,
    saving_path = folder_path,
    device = 'cuda:0',
    torch_or_numpy = 'torch',
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.14it/s]


Trajectory 0 has been saved to /project/dinner/zpengmei/geom2vec/results/chignolin_stride10 using torch.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.68it/s]


Trajectory 1 has been saved to /project/dinner/zpengmei/geom2vec/results/chignolin_stride10 using torch.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.47it/s]


...

Trajectory 51 has been saved to /project/dinner/zpengmei/geom2vec/results/chignolin_stride10 using torch.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.63it/s]


Trajectory 52 has been saved to /project/dinner/zpengmei/geom2vec/results/chignolin_stride10 using torch.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.78it/s]

Trajectory 53 has been saved to /project/dinner/zpengmei/geom2vec/results/chignolin_stride10 using torch.


In [1]:
# Here is how you load and create a dataset
from geom2vec.data import Preprocessing

folder_path = '/project/dinner/zpengmei/geom2vec/results/chignolin_stride10'
preprocess = Preprocessing(torch_or_numpy='torch')
raw_dataset = preprocess.load_dataset(data_path=folder_path,mmap_mode='r')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:03<00:00, 14.84it/s]


In [2]:
raw_dataset[0].shape

torch.Size([1000, 10, 4, 128])

In [3]:
# And you can create a time-lagged dataset like this
time_lagged_dataset = preprocess.create_dataset(raw_dataset, lag_time=10)
print(time_lagged_dataset[0][0].shape)
print(time_lagged_dataset[0][1].shape)

torch.Size([10, 4, 128])
torch.Size([10, 4, 128])
